In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from numba import njit

## Naive loop

In [28]:
%%time
def plist(amax):
    l = []
    for a in np.arange(1,amax):
        s = float((3*a + 1)/2)
        A = np.sqrt(s*(s-a)*(s-a)*(s-(a+1)))
        if np.abs(A - np.round(A))<0.0000001:
            l.append([a,a,a+1])
    return l
plist(100)

CPU times: user 2.19 ms, sys: 26 µs, total: 2.22 ms
Wall time: 2.21 ms


[[1, 1, 2], [5, 5, 6], [65, 65, 66]]

In [22]:
a = 995333
s = (3*a + 1)/2
A = np.sqrt(s*(s-a)*(s-a)*(s-(a+1)))
A

428980680062.0

## Using Pell's equation

In [89]:
## Continued fraction expansion of square root of d

def heron(a,b,c):
    s = (a+b+c)/2
    return np.sqrt(s*(s-a)*(s-b)*(s-c))

def CF(d,n):
    if np.sqrt(d) != np.floor(np.sqrt(d)):
        
        a = []
        r = []
        u = []
        v = []
        h = []
        k = []
    
        a += [int(np.floor(np.sqrt(d)))]
        u += [a[0]]
        v += [1]
        r += [(np.sqrt(d) - u[0])/v[0]]
    
        a += [int(np.floor(1/r[0]))]
        v += [int((d - u[0]*u[0])/v[0])]
        u += [a[1]*v[1] - u[0]]
        r += [(np.sqrt(d) - u[1])/v[1]]
    
        h+=[a[0]]
        k+=[1]
        h+=[a[1]*h[0] + 1]
        k+=[a[1]*k[0]]
    
        for i in np.arange(2, n):
            a += [int(np.floor(1/r[i-1]))]
            v += [int((d - u[i-1]*u[i-1])/v[i-1])]
            u += [a[i]*v[i] - u[i-1]]
            r += [(np.sqrt(d) - u[i])/v[i]]
            h += [a[i]*h[i-1] + h[i-2]]
            k += [a[i]*k[i-1] + k[i-2]]       
    
        res  = {}
        res["a"] = a
        res["h"] = h
        res["k"] = k
        
    return res

In [82]:
res = CF(3, 40)
l = res["k"]
n = res["h"]

a1_list = []
for i in np.arange(len(n)):
    if n[i]**2 - 3*l[i]**2 == 1:
        if (-1 + n[i])%3==0:
            a1_list += [2*(-1 + n[i])//3 + 1]

a2_list = []
for i in np.arange(len(n)):
    if n[i]**2 - 3*l[i]**2 == 1:
        if (-2 + n[i])%3==0:
            a2_list += [2*(-2 + n[i])//3 + 1]

In [98]:
tr_df_list=[]

for a in a1_list:
    if 3*a+1 <= 1e9:
        p = (3*a+1)
        A = heron(a,a,a+1)
        tr_df_list.append(pd.DataFrame({"a":[a], "p":p, "A":A}))

for a in a2_list:
    if 3*a+1 <= 1e9:
        p = (3*a-1)
        A = heron(a,a,a-1)
        tr_df_list.append(pd.DataFrame({"a":[a], "p":p, "A":A}))

tr_df = pd.concat(tr_df_list)
tr_df["p"].sum()

518408348

In [99]:
tr_df

,a,p,A
0,5,16,1.200000e+01
0,65,196,1.848000e+03
0,901,2704,3.517800e+05
0,12545,37636,6.814987e+07
0,174725,524176,1.321942e+10
0,2433601,7300804,2.564481e+12
0,33895685,101687056,4.974959e+14
0,1,2,0.000000e+00
0,17,50,1.200000e+02
0,241,722,2.508000e+04
